<a href="https://colab.research.google.com/github/cmeneses1/GeokMedoidsCalculator/blob/main/kMedoids_Allande1773.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# kMedoids for villages in Allande at 1773
----------
We're going to apply kMedians algorithm for a complete set of locations of villages from Allande's Council in 1773. There are two distances to be applied: one is the traveling (by car) distances in kilometres and, the other one, the traveling time by car in minutes. The number of clusters has been chosen as 18, because we want to compare our computed clustering with historical clustering by parishes.

## 1. Installing and importing dependencies
Install this required package.

In [ ]:
!pip install scikit-learn-extra

     |████████████████████████████████| 1.7 MB 17.4 MB/s 


Import those required packages.

In [ ]:
from geopy import distance
from sklearn_extra.cluster import KMedoids
from seaborn import color_palette
import matplotlib.pyplot as plt
import folium
import pandas as pd
import numpy as np
import requests
import json

## 2. Importing data
Loading the villages data. The villages source is a census transcribed by [Antonio García Linares](https://www.edicioneshidalguia.es/?product=revista-no-240-8-linajes-asturianos-padrones-del-concejo-de-allande-de-1698-y-1773) and their geo-locations have been caught from [pueblos de Asturias](https://www.pueblosdeasturias.es/) and [Open Street Map](https://www.openstreetmap.org/#map=6/42.188/2.878).

In [ ]:
#@title
%%file test.txt
Name	Latitude	Longitude	Parish	ParishVillage
Parajas	43.2016456	-6.585278	Araniego (Pajaras)	1
Argancinas	43.2064324	-6.5874077	Araniego (Pajaras)	0
Berducedo	43.2329138	-6.7676724	Berducedo	1
Trapa	43.2566454	-6.7852052	Berducedo	0
Teijedo	43.2545971	-6.7749775	Berducedo	0
Castello	43.2529118	-6.7626349	Berducedo	0
Baldedo	43.2480584	-6.7673603	Berducedo	0
Castro, El	43.2702311	-6.7512008	Berducedo	0
Grandera, La	43.268669	-6.774232	Berducedo	0
Corondeño	43.2636188	-6.7713329	Berducedo	0
Forniellas	43.1999761	-6.6282732	Besullo	0
Fuentes	43.19679	-6.616097	Besullo	0
Noceda	43.1944265	-6.6178032	Besullo	0
Iboyo	43.2108331	-6.6354339	Besullo	0
Comba	43.2042325	-6.6444941	Besullo	0
Besullo	43.1869801	-6.6329396	Besullo	1
Celón	43.2402821	-6.5825589	Celón	1
Vega de Truelles, La	43.2278243	-6.5844473	Celón	0
Pumar	43.2305154	-6.5885858	Celón	0
Herías	43.3154383	-6.805375	Herías	1
Sarzol	43.3261506	-6.8221276	Herías	0
Navedo	43.3197513	-6.7777945	Herías	0
Río de Villar	43.321479	-6.800871	Herías	0
Lago	43.2533629	-6.7321047	Lago	1
Montefurado	43.2660529	-6.6946773	Lago	0
Carcedo de Lago	43.2420634	-6.736366	Lago	0
Carcedo de Lomes	43.2141137	-6.5701844	Lomes	0
Armenande	43.2344383	-6.7415478	Lago	0
Villardejusto	43.2287387	-6.7410068	Lago	0
Villar de Castanedo	43.2675174	-6.7116478	Lomes	0
Castanedo	43.2675174	-6.7116478	Lago	0
San Pedro	43.2673257	-6.7248198	Lago	0
Linares	43.233823	-6.562501	Linares	1
Arganzúa	43.2381359	-6.5405136	Linares	0
Lomes	43.2141137	-6.5701844	Lomes	1
Otero	43.2260237	-6.5622274	Lomes	0
Tarallé	43.2169972	-6.5784209	Lomes	0
Pola de Allande	43.271754	-6.611161	Pola de Allande	1
Reigada, La	43.291497	-6.657195	Pola de Allande	0
Peñaseita	43.2826	-6.6352	Pola de Allande	0
Colobredo	43.2822369	-6.6283227	Pola de Allande	0
Mazo, El	43.2728346	-6.6205657	Pola de Allande	0
Fresnedo	43.266826	-6.619494	Pola de Allande	0
Fresnedo de San Emiliano	43.2574581	-6.8336809	San Emiliano	0
Cereceda	43.2578766	-6.6035813	Pola de Allande	0
Valbona	43.2617475	-6.590518	Pola de Allande	0
Villafrontú	43.2719865	-6.5982218	Pola de Allande	0
Ferroy	43.2799489	-6.6033123	Pola de Allande	0
Caleyo, El	43.271754	-6.611161	Pola de Allande	0
Cimadevilla	43.27484	-6.61511	Pola de Allande	0
Presnas	43.2295493	-6.5761443	Presnas	1
San Emiliano	43.2574581	-6.8336809	San Emiliano	1
Murias	43.2473996	-6.832618	San Emiliano	0
Trigaledo, El	43.2473996	-6.832618	San Emiliano	0
Vallinas	43.2655327	-6.8253966	San Emiliano	0
Ema	43.263044	-6.8201096	San Emiliano	0
Quintana, La	43.2675674	-6.8155934	San Emiliano	0
Villadecabo	43.266058	-6.8096113	San Emiliano	0
Buslavín	43.2557835	-6.7996059	San Emiliano	0
Figuerina, La	43.2345953	-6.7999155	San Emiliano	0
Beveraso	43.2781441	-6.7986945	San Emiliano	0
Bojo	43.2788464	-6.7843805	San Emiliano	0
Tamagordas	43.2910992	-6.8251577	San Emiliano	0
Cernías	43.2996843	-6.8277327	San Emiliano	0
Estela	43.2950755	-6.7974456	San Emiliano	0
Riodecoba	43.3073637	-6.7926056	San Emiliano	0
San Martín de Beduledo	43.2361028	-6.5938196	San Martín de Beduledo	1
Santa Coloma	43.2960698	-6.7252148	Santa Coloma	1
Cabral	43.3020667	-6.7111294	Santa Coloma	0
Sellón, El	43.2925072	-6.7263337	Santa Coloma	0
Meres	43.292	-6.7253	Santa Coloma	0
Bustel	43.2893098	-6.721016	Santa Coloma	0
Pontenova	43.27599	-6.73081	Santa Coloma	0
Arbeyales	43.28138	-6.72776	Santa Coloma	0
Vallinadosa	43.29101	-6.71304	Santa Coloma	0
Llances	43.28506	-6.74181	Santa Coloma	0
Monón	43.2946943	-6.737048	Santa Coloma	0
Is	43.28503	-6.75724	Santa Coloma	0
Bendón	43.2999849	-6.7436019	Santa Coloma	0
Muriellos	43.3053548	-6.737544	Santa Coloma	0
Rebollo, El	43.3215955	-6.7111464	Santa Coloma	0
Folgueriza, La	43.3482941	-6.6910096	Santa Coloma	0
Bustantigo	43.359591	-6.6895435	Santa Coloma	0
Porquera, La	43.31615	-6.72182	Santa Coloma	0
Plantao, El	43.36269	-6.6858	Santa Coloma	0
San Martín del Valledor	43.1779708	-6.7741057	San Martín del Valledor	1
Robledo	43.1855984	-6.7654259	San Martín del Valledor	0
Villasonte	43.19069	-6.765	San Martín del Valledor	0
Salcedo	43.2013551	-6.7615475	San Martín del Valledor	0
Engertal, El	43.2071312	-6.7579963	San Martín del Valledor	0
Provo, El	43.2169103	-6.7527332	San Martín del Valledor	0
Tremado	43.1700598	-6.7903301	San Martín del Valledor	0
Cornollo	43.17112	-6.8165671	San Martín del Valledor	0
Paradas	43.1885	-6.8179	San Martín del Valledor	0
Busvidal	43.2005095	-6.8174364	San Martín del Valledor	0
Coba	43.177474	-6.7657864	San Martín del Valledor	0
Furada, La	43.1713428	-6.7583648	San Martín del Valledor	0
Rubieiro	43.1684703	-6.771065	San Martín del Valledor	0
Aguanes	43.1558756	-6.7692736	San Martín del Valledor	0
Trabaces	43.1478821	-6.771129	San Martín del Valledor	0
Collada	43.15628	-6.792	San Martín del Valledor	0
Barras	43.14939	-6.79432	San Martín del Valledor	0
Villanueva	43.1440866	-6.7949573	San Martín del Valledor	0
Fonteta	43.13617	-6.80743	San Martín del Valledor	0
Villalaín	43.1224534	-6.8133427	San Martín del Valledor	0
San Salvador	43.15547	-6.805	San Martín del Valledor	0
Bustarel	43.16078	-6.83129	San Martín del Valledor	0
Villagrufe	43.2442678	-6.5995361	Villagrufre	1
Tamuño	43.25211	-6.59119	Villagrufre	0
Carballedo	43.24076	-6.60469	Villagrufre	0
Pradiella	43.2400293	-6.618026	Villagrufre	0
Santullano	43.24249	-6.61762	Villagrufre	0
Prada	43.2384413	-6.6287743	Villagrufre	0
Villar de Sapos	43.2039077	-6.5953287	Villar de Sapos	1
Selce	43.20107	-6.60395	Villar de Sapos	0
Almoño	43.20816	-6.60055	Villar de Sapos	0
Villavaser	43.2437755	-6.578371	Villavaser	1
Piniella	43.24747	-6.58049	Villavaser	0
Figueras	43.2540035	-6.5776323	Villavaser	0
Villaverde	43.2262624	-6.6087403	Villaverde	1
Lantigo	43.2264554	-6.6153677	Villaverde	0
Valle, El	43.22284	-6.61701	Villaverde	0
Abaniella	43.219795	-6.6159873	Villaverde	0
Peruyeda	43.2124106	-6.6267596	Villaverde	0
Santa Eulalia	43.2239258	-6.598798	Villaverde	0

Writing test.txt


In [ ]:
name = 'test.txt'
data = pd.read_table(name)
data

,Name,Latitude,Longitude,Parish,ParishVillage
0,Parajas,43.201646,-6.585278,Araniego (Pajaras),1
1,Argancinas,43.206432,-6.587408,Araniego (Pajaras),0
2,Berducedo,43.232914,-6.767672,Berducedo,1
3,Trapa,43.256645,-6.785205,Berducedo,0
4,Teijedo,43.254597,-6.774977,Berducedo,0
...,...,...,...,...,...
120,Lantigo,43.226455,-6.615368,Villaverde,0
121,"Valle, El",43.222840,-6.617010,Villaverde,0
122,Abaniella,43.219795,-6.615987,Villaverde,0
123,Peruyeda,43.212411,-6.626760,Villaverde,0


## 3. Arrays of traveling distance
Now, we calculate both distance arrays from [OSMR API](http://project-osrm.org/docs/v5.5.1/api/?language=cURL#general-options).

In [ ]:
#@title
# Longitude and Latitude vectors
Longitude = data.Longitude
Latitude = data.Latitude

# Number of locations
n = len(Longitude)

# Total distances and durations array
distancesT = np.zeros((125,125), dtype=float)
durationsT = np.zeros((125,125), dtype=float)

for k in range(0, 5):
    for l in range(k+1, 5):
        # Creating a convenient string for OSRM service
        lonLatString = ''
        for i in range(0+k*25, 25+k*25):
            lon = Longitude[i]
            lat = Latitude[i]
            lonLatString += str(lon) + ',' + str(lat) + ';'
        for i in range(0+l*25, 25+l*25):
            lon = Longitude[i]
            lat = Latitude[i]
            lonLatString += str(lon) + ',' + str(lat) + ';'

         # Not interested in the last ';' string
        lonLatString = lonLatString[0:-1]

        # call the OSMR API
        osmrString = "http://router.project-osrm.org/table/v1/driving/"\
                + lonLatString + '?annotations=distance,duration'
        r = requests.get(osmrString)

        # Extracting driving distance array, in kilometres, and making it symmetrical
        distances = 1/1000 * np.array(json.loads(r.content)['distances'])
        distances = 1/2 * (distances + np.transpose(distances))
        if k == (l-1):
            distancesT[(0+k*25):(25+k*25), (0+k*25):(25+k*25)] = distances[0:25, 0:25]
        distances = distances[0:25, 25:50]
        distancesT[(0+k*25):(25+k*25), (0+l*25):(25+l*25)] = distances
        distancesT[(0+l*25):(25+l*25), (0+k*25):(25+k*25)] = np.transpose(distances)

        # Extracting driving durations array, in minutes, and making it symmetrical
        durations = 1/1000 * np.array(json.loads(r.content)['durations'])
        durations = 1/2 * (durations + np.transpose(durations))
        if k == (l-1):
            durationsT[(0+k*25):(25+k*25), (0+k*25):(25+k*25)] = durations[0:25, 0:25]
        durations = durations[0:25, 25:50]
        durationsT[(0+k*25):(25+k*25), (0+l*25):(25+l*25)] = durations
        durationsT[(0+l*25):(25+l*25), (0+k*25):(25+k*25)] = np.transpose(durations)

# There is one case left, k = l = 4.
k = 4
# Creating a convenient string for OSRM service
lonLatString = ''
for i in range(0+k*25, 25+k*25):
    lon = Longitude[i]
    lat = Latitude[i]
    lonLatString += str(lon) + ',' + str(lat) + ';'

# Not interested in the last ';' string
lonLatString = lonLatString[0:-1]

# call the OSMR API
osmrString = "http://router.project-osrm.org/table/v1/driving/"\
        + lonLatString + '?annotations=distance,duration'
r = requests.get(osmrString)

# Extracting driving distance array, in kilometres, and making it symmetrical
distances = 1/1000 * np.array(json.loads(r.content)['distances'])
distances = 1/2 * (distances + np.transpose(distances))
distancesT[(0+k*25):(25+k*25), (0+k*25):(25+k*25)] = distances[0:25, 0:25]

# Extracting driving durations array, in minutes, and making it symmetrical
durations = 1/1000 * np.array(json.loads(r.content)['durations'])
durations = 1/2 * (durations + np.transpose(durations))
durationsT[(0+k*25):(25+k*25), (0+k*25):(25+k*25)] = durations[0:25, 0:25]

In [ ]:
#@title
# Geodesic distances array
geodesicT = np.zeros((125,125), dtype=float)

# Calculating distances from geopy function `distance`in kilometres
for i in range(0, n):
    t1 = (Latitude[i], Longitude[i])

    for j in range(0, n):
        t2 = (Latitude[j], Longitude[j])

        if i < j:
            geodesicT[i, j] = distance.distance(t1, t2).km
            geodesicT[j, i] = geodesicT[i, j]

# 4. The kMedoids algorithm and map representation. Distances array
Lets compute the `KMedoids` algorithm for the distances in kilometres.

But, first, this functions calculates the whithin cluster sum of distances.

In [ ]:
#@title
def sumDecomposition(distances, medoids, labels):
    """
    This function calculates the total sum of distances, T, the sum of distances 
    within clusters, W, and the sum of distances between clusters, B.
    Arguments:
        distances: array of distances.
        medoids: list of medoid indices.
        labels: list of clustering labels.
    """
    T = 1/2 * np.sum(distances)
    W = 0
    B = 0
    for cluster in range(0, len(medoids)):
        for i, elem in enumerate(labels):
            if elem == cluster:
                for j, elem2 in enumerate(labels):
                    if elem2 == cluster:
                        W += distances[i, j]
                    else:
                        B += distances[i, j]
    W *= 1/2
    B *= 1/2
    return T, W, B

In [ ]:
#@title
# Choose a number of cluster
n_clusters = 18

# Apply KMedoids function.
kmedoids = KMedoids(n_clusters=n_clusters,
                    metric='precomputed',
                    init='k-medoids++',
                    method='pam').fit(distancesT)
medoids = kmedoids.medoid_indices_
labels = kmedoids.labels_
T, W, B = sumDecomposition(distancesT, medoids, labels)
print('Medoid indices:', medoids)
print('Medoid labels:', labels)
print(f"T = {T}, W = {W}, B = {B}, W/T = {W/T}")

Medoid indices: [ 69  52  26  97  23 105  19  61  48  94   2 121 110 114  65  82  85  50]
Medoid labels: [13  2 10 10 10 10 10  0  4  4 13 13 13 11 13 13 17 17 17  6  6 14  6  4
  4  4  2  4  4  4  4  4 17 17  2  2  2  8  8  8  8  8  8  1  8  8  8  8
  8  8 17  1  1  1  1  1  1  1  1  9  7  7 14  6 14 14 17  0  0  0  0  0
  0  0  4  0  0  7  0  0  0 15 15  0 15 16 16 16 16 16 10 16 16  9  9  3
  3  3  3  3  5  5  5  5  5  5 16 12  8 12 12 12 12 13 13 13 17 17  8 11
 11 11 11 11 11]
T = 228543.62465, W = 2314.7958000000003, B = 226228.82884999987, W/T = 0.010128463673160703


Lets represent in a map.

In [ ]:
#@title
# Creates the map
f = folium.Figure(width='65%')
m = folium.Map(location=[Latitude.mean(), Longitude.mean()]).add_to(f)

# Having a touch of color.
color = color_palette("husl", n_clusters).as_hex()

# Representing our clustering medoids
for i, elem in enumerate(medoids):
    folium.Circle(
        location=[Latitude[elem], Longitude[elem]],
        radius=600,
        color=color[i],
        fill=False,
        fill_color=color[i],
    ).add_to(m)

# Representing our clustering output
for i, elem in enumerate(labels):
    folium.Circle(
        location=[Latitude[i], Longitude[i]],
        radius=200,
        popup=f'{i}: ' + data.Name[i]+'. '+ f'Medoid {medoids[elem]}.',
        color=color[elem],
        fill=True,
        fill_color=color[elem],
    ).add_to(m)

# Adjust zoom
sw = data[['Latitude', 'Longitude']].min().values.tolist()
ne = data[['Latitude', 'Longitude']].max().values.tolist()
m.fit_bounds([sw, ne]) 

m

# 5. The kMedoids algorithm and map representation. Durations array
Lets compute the `KMedoids` algorithm for the distances in minutes.

In [ ]:
#@title
# Choose a number of cluster
n_clusters = 18

# Apply KMedoids function.
kmedoidsTime = KMedoids(n_clusters=n_clusters,
                    metric='precomputed',
                    init='k-medoids++',
                    method='pam').fit(durationsT)
medoidsTime = kmedoidsTime.medoid_indices_
labelsTime = kmedoidsTime.labels_
TTime, WTime, BTime = sumDecomposition(durationsT, medoidsTime, labelsTime)
print('Medoid indices:', medoidsTime)
print('Medoid labels:', labelsTime)
print(f"T = {TTime}, W = {WTime}, B = {BTime}, W/T = {WTime/TTime}")

Medoid indices: [ 37  43  27  97 105 114   2  82  69  19  61  32 119  91  21  29 110  74]
Medoid labels: [ 5  5  6  6  6  6  6  8  6  6  5  5  5 12  5  5 11 12 12  9  9 14  9  6
  6  2 11  2  2 15 15  6 11 11 11 11 11  0  0  0  0  0  0  1  0  0  0  0
  0  0 11  1  1  1  1  1  1  1  1  6 10 10  1  9  9 14 11  8  8  8  8  8
  8  8 17  8  8  8  8  8  8  7  7  8  7 13 13 13  6  6  6 13 13 13  6  3
  3  3  3  3  4  4  4  4  4  4 13 16  0 16 16 16 16  5  5  5 11 11 11 12
 12 12 12 12 12]
T = 19008.4356, W = 365.51324999999974, B = 18642.922350000044, W/T = 0.019229002201527817


In [ ]:
#@title
# Creates the map
f = folium.Figure(width='65%')
m = folium.Map(location=[Latitude.mean(), Longitude.mean()]).add_to(f)

# Having a touch of color.
color = color_palette("husl", n_clusters).as_hex()

# Representing our clustering medoids
for i, elem in enumerate(medoidsTime):
    folium.Circle(
        location=[Latitude[elem], Longitude[elem]],
        radius=600,
        color=color[i],
        fill=False,
        fill_color=color[i],
    ).add_to(m)

# Representing our clustering output
for i, elem in enumerate(labelsTime):
    folium.Circle(
        location=[Latitude[i], Longitude[i]],
        radius=200,
        popup=f'{i}: ' + data.Name[i]+'. '+ f'Medoid {medoidsTime[elem]}.',
        color=color[elem],
        fill=True,
        fill_color=color[elem],
    ).add_to(m)

# Adjust zoom
sw = data[['Latitude', 'Longitude']].min().values.tolist()
ne = data[['Latitude', 'Longitude']].max().values.tolist()
m.fit_bounds([sw, ne]) 

m

# 6. The kMedoids algorithm and map representation. Geodesic distance
Lets compute the `KMedoids` algorithm for the geodesic distances in kilometres.

In [ ]:
#@title
# Choose a number of cluster
n_clusters = 18

# Apply KMedoids function.
kmedoidsGeod = KMedoids(n_clusters=n_clusters,
                    metric='precomputed',
                    init='k-medoids++',
                    method='pam').fit(geodesicT)
medoidsGeod = kmedoidsGeod.medoid_indices_
labelsGeod = kmedoidsGeod.labels_
TGeod, WGeod, BGeod = sumDecomposition(geodesicT, medoidsGeod, labelsGeod)
print('Medoid indices:', medoidsGeod)
print('Medoid labels:', labelsGeod)
print(f"T = {TGeod}, W = {WGeod}, B = {BGeod}, W/T = {WGeod/TGeod}")

Medoid indices: [ 37  56  95 120  69  16  39 101  82   9  10  89  93  30  19  52  27   1]
Medoid labels: [17 17 16  9  9  9  9  9  9  9 10 10 10 10 10 10  5  5  5 14 14 14 14 16
 13 16 17 16 16 13 13 13  5  5 17  5 17  0  6  6  6  0  0 15  0  0  0  0
  0  0  5 15 15 15  1  1  1  1  1 15  1  9  1 14 14 14  5  4  4  4  4  4
 13  4  4  4  4  9  4  4  4  8  8  4  8  2  2  2 11 11 11  2 12 12 12  2
  2  2  7  7  7  7  7  7  7  7  7  5  5  5  3  3  3 17 17 17  5  5  5  3
  3  3  3 10  3]
T = 87986.40558861144, W = 896.8513256284728, B = 87089.5542629825, W/T = 0.010193066981526486


In [ ]:
#@title
# Creates the map
f = folium.Figure(width='65%')
m = folium.Map(location=[Latitude.mean(), Longitude.mean()]).add_to(f)

# Having a touch of color.
color = color_palette("husl", n_clusters).as_hex()

# Representing our clustering medoids
for i, elem in enumerate(medoidsGeod):
    folium.Circle(
        location=[Latitude[elem], Longitude[elem]],
        radius=600,
        color=color[i],
        fill=False,
        fill_color=color[i],
    ).add_to(m)

# Representing our clustering output
for i, elem in enumerate(labelsGeod):
    folium.Circle(
        location=[Latitude[i], Longitude[i]],
        radius=200,
        popup=f'{i}: ' + data.Name[i]+'. '+ f'Medoid {medoidsGeod[elem]}.',
        color=color[elem],
        fill=True,
        fill_color=color[elem],
    ).add_to(m)

# Adjust zoom
sw = data[['Latitude', 'Longitude']].min().values.tolist()
ne = data[['Latitude', 'Longitude']].max().values.tolist()
m.fit_bounds([sw, ne]) 

m

# 7. Clustering by parishes

In [ ]:
#@title
# List of parishes and parish villages 
parishes = data.Parish
parishVillages = data.ParishVillage

# Labels by parish
parishLabel = []
dic = {}
count = -1
for i in parishes:
    if dic.get(i) == None:
        count += 1
        dic[i] = count
        parishLabel.append(count)
    else:
        parishLabel.append(dic.get(i))

# Creates the map
f = folium.Figure(width='65%')
m = folium.Map(location=[Latitude.mean(), Longitude.mean()]).add_to(f)

# Having a touch of color.
color = color_palette("husl", n_clusters).as_hex()

# Representing our clustering medoids
for i, elem in enumerate(parishVillages):
    if elem == 1:
        folium.Circle(
            location=[Latitude[i], Longitude[i]],
            radius=600,
            color=color[parishLabel[i]],
            fill=False,
            fill_color=color[parishLabel[i]],
        ).add_to(m)

# Representing our clustering output
for i, elem in enumerate(parishLabel):
    folium.Circle(
        location=[Latitude[i], Longitude[i]],
        radius=200,
        popup=f'{i}: ' + data.Name[i]+'. '+ f'Medoid {parishes[i]}.',
        color=color[elem],
        fill=True,
        fill_color=color[elem],
    ).add_to(m)

# Adjust zoom
sw = data[['Latitude', 'Longitude']].min().values.tolist()
ne = data[['Latitude', 'Longitude']].max().values.tolist()
m.fit_bounds([sw, ne]) 

m